In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
!pip3 install einops
!pip3 install pillow
!pip3 install tqdm

In [ ]:
from torch.utils import data
import numpy as np

In [ ]:
%cd /content/DDPM_MP/denoising_diffusion_pytorch
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Trainer, Dataset_traj, cycle, num_to_groups
import torch
from torch import nn
device = torch.device("cuda")
# define the U-net structure
model = Unet(
    dim = 32,
    dim_mults = (1, 2, 2, 4 ),
    groups = 8
).to(device)
model = nn.DataParallel(model)
model.to(device)

# define diffusion model
op_num = 1
konw_op_num = 0
diffusion = GaussianDiffusion(
    model,                        # U-net model
    timesteps = 1000,             # number of diffusion steps
    unmask_number=konw_op_num+1,  # the dimension of x2 in P(x1|x2)
    loss_type = 'l2'              # L1 or L2
).to(device)

#set training parameters
trainer = Trainer(
    diffusion,                                   # diffusion model
    folder = '/content',                        # folder of trajectories
    system = 'salm_10',  # name of the trajectory, the file {system}_traj.npy will be used as the training set
    train_batch_size = 128,                      # training batch size
    train_lr = 1e-5,                             # learning rate
    train_num_steps = 100001,                   # total training steps
    gradient_accumulate_every = 1,               # gradient accumulation steps
    ema_decay = 0.995,                           # exponential moving average decay
    op_number = op_num,
    fp16 = False                                 # turn on mixed precision training with apex
)
trainer.train()


In [ ]:
batch_size = 1000  #the number of samples generated in each batch
sample_ds = Dataset_traj('/content/', 'z_predict')
sample_ds.max_data = trainer.ds.max_data
sample_ds.min_data = trainer.ds.min_data        #To ensure that the sample data is scaled in the same way as the training data
sample_dl = cycle(data.DataLoader(sample_ds, batch_size = batch_size, shuffle=True, pin_memory=True))
T_sample = np.load('/content/z_predict_traj.npy')
print(T_sample)
print(np.shape(T_sample))

In [ ]:
%cd /content/DDPM_MP/denoising_diffusion_pytorch
num_sample = 77407  #total number of samples
model_id = 4
trainer.load(model_id)

batches = num_to_groups(num_sample, batch_size)
all_ops_list = list(map(lambda n: trainer.ema_model.sample(trainer.op_number, batch_size=n, samples = next(sample_dl).cuda()[:n, :]), batches))
all_ops = torch.cat(all_ops_list, dim=0).cpu()
all_ops = trainer.rescale_sample_back(all_ops)
np.save('/content/DDPM_MP/denoising_diffusion_pytorch/results/salm_10_final.npy', all_ops.numpy())

In [ ]:
!zip -r /content/salm_10.zip /content/DDPM_MP


In [ ]:
from google.colab import files
files.download("/content/salm_10.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>